# Model Rekomondasi

### Persiapaan Data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import hstack


In [2]:
def prepare_datasets(corpus, labels, test_data_proportion=0.3, random_state=42):
    train_X, test_X, train_Y, test_Y = train_test_split(corpus, labels,
                                                        test_size=test_data_proportion,
                                                        random_state=random_state)
    return train_X, test_X, train_Y, test_Y
    
def tfidf_extractor(corpus, ngram_range=(1,2)):
    vectorizer = TfidfVectorizer(min_df=1,
                                norm='l2',
                                smooth_idf=True,
                                use_idf=True,
                                ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

def bow_extractor(corpus, ngram_range=(1,1)):
    vectorizer = CountVectorizer(min_df=1, ngram_range=ngram_range)
    features = vectorizer.fit_transform(corpus)
    return vectorizer, features

In [3]:
df = pd.read_csv('sentiment_otomatis_3000.csv')
df.head(10)

,content,sentimen
0,ivan 31 pedagang sayur pasar kemiri muka depok...,negatif
1,pasukan militer israel operasi serangan tepi b...,negatif
2,suami berinisial 32 tega kekerasan rumah tangg...,negatif
3,menghambat pasokan bantuan penduduk gaza kejah...,negatif
4,praktik dunia kerap ditemukan perusahaan menah...,negatif
5,cawapres nomor urut 1 muhaimin iskandar cak im...,negatif
6,ketum pdi perjuangan pdip megawati soekarnoput...,negatif
7,pengguna lrt jabodebek mengeluhkan waktu tungg...,negatif
8,heboh media sosial isu anggota bem universitas...,negatif
9,mobil dipamerkan mal semarang jawa tengah hila...,negatif


In [4]:
# Split data
x_train, x_test, y_train, y_test = prepare_datasets(df['content'], df['sentimen'], test_data_proportion=0.3)


In [5]:
# Convert text labels to numerical labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Ensure the lengths match after splitting
y_train_encoded = y_train_encoded[:len(x_train)]
y_test_encoded = y_test_encoded[:len(x_test)]


In [6]:
# TF-IDF Vectorization
tfidf_vectorizer, tfidf_train_features = tfidf_extractor(x_train)
tfidf_test_features = tfidf_vectorizer.transform(x_test)

# Bag of Words (BoW) features
count_vectorizer, count_train_features = bow_extractor(x_train)
count_test_features = count_vectorizer.transform(x_test)

# Combine TF-IDF and BoW features
combined_train_features = hstack([tfidf_train_features, count_train_features])
combined_test_features = hstack([tfidf_test_features, count_test_features])

In [7]:
# TF-IDF Features
print("Jumlah fitur TF-IDF:", tfidf_train_features.shape[1])

# Bag of Words (BoW) Features
print("Jumlah fitur Bag of Words:", count_train_features.shape[1])

# Combined Features
print("Jumlah fitur gabungan:", combined_train_features.shape[1])

Jumlah fitur TF-IDF: 136955
Jumlah fitur Bag of Words: 18458
Jumlah fitur gabungan: 155413


### Model Naiye Bayes

In [8]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
# Define function to train and evaluate Naive Bayes classifier
def train_and_evaluate(classifier, train_features, train_labels, test_features, test_labels):
    # Train the classifier
    classifier.fit(train_features, train_labels)
    
    # Predictions
    predictions = classifier.predict(test_features)
    
    # Calculate accuracy
    accuracy = accuracy_score(test_labels, predictions)
    
    # Calculate F1 score
    f1 = f1_score(test_labels, predictions, average='weighted')
    
    # Classification report
    report = classification_report(test_labels, predictions, target_names=label_encoder.classes_)
    
    # Confusion matrix
    matrix = confusion_matrix(test_labels, predictions)
    
    return accuracy * 100, f1 * 100, report, matrix

In [10]:
# Function to plot confusion matrix
def plot_confusion_matrix(matrix, labels):
    plt.figure(figsize=(8, 6))
    sns.heatmap(matrix, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.show()

#### Tanpa Parameter

In [11]:
# Initialize Naive Bayes classifier
model_naiye_tanpa_parameter = MultinomialNB()

# Train and evaluate on Bag of Words (BoW) features
bow_accuracy, bow_f1,bow_report,bow_matrix = train_and_evaluate(model_naiye_tanpa_parameter , count_train_features, y_train_encoded, count_test_features, y_test_encoded)




In [12]:
print("Bag of Words (BoW) Features:")
print("Accuracy: {:.2f}%".format(bow_accuracy))
print("F1 Score: {:.2f}%".format(bow_f1))
print("Classification Report:\n", bow_report)
print("Confusion Matrix:\n", bow_matrix)

Bag of Words (BoW) Features:
Accuracy: 86.56%
F1 Score: 86.42%
Classification Report:
               precision    recall  f1-score   support

     negatif       0.87      0.86      0.87       313
      netral       0.93      0.76      0.84       291
     positif       0.81      0.97      0.88       296

    accuracy                           0.87       900
   macro avg       0.87      0.87      0.86       900
weighted avg       0.87      0.87      0.86       900

Confusion Matrix:
 [[270  13  30]
 [ 33 222  36]
 [  6   3 287]]


### Tuning GridCV

In [14]:
# Define the parameter grid
param_grid = {'alpha': [0.1, 0.5, 1.0, 5.0, 10.0]}

# Initialize GridSearchCV
grid_search = GridSearchCV(model_naiye_tanpa_parameter, param_grid, cv=5, scoring='accuracy')

# Fit the grid search to the data
grid_search.fit(count_train_features, y_train_encoded)

# Get the best parameter
best_alpha = grid_search.best_params_['alpha']
print("Best alpha:", best_alpha)

# Get the best model
best_model = grid_search.best_estimator_
print("Best alpha:", best_model)

Best alpha: 0.1
Best alpha: MultinomialNB(alpha=0.1)


#### Pakai Paramater

In [15]:
# Initialize Naive Bayes classifier
model_naiye_parameter = MultinomialNB(alpha=0.1)


# Train and evaluate on Bag of Words (BoW) features
bow_accuracy, bow_f1,bow_report,bow_matrix = train_and_evaluate(model_naiye_parameter , count_train_features, y_train_encoded, count_test_features, y_test_encoded)


In [16]:
print("Bag of Words (BoW) Features:")
print("Accuracy: {:.2f}%".format(bow_accuracy))
print("F1 Score: {:.2f}%".format(bow_f1))
print("Classification Report:\n", bow_report)
print("Confusion Matrix:\n", bow_matrix)

Bag of Words (BoW) Features:
Accuracy: 87.89%
F1 Score: 87.80%
Classification Report:
               precision    recall  f1-score   support

     negatif       0.87      0.87      0.87       313
      netral       0.93      0.80      0.86       291
     positif       0.85      0.96      0.90       296

    accuracy                           0.88       900
   macro avg       0.88      0.88      0.88       900
weighted avg       0.88      0.88      0.88       900

Confusion Matrix:
 [[273  14  26]
 [ 33 233  25]
 [  8   3 285]]


### Model SVM

In [17]:
from sklearn.svm import SVC

#### Tanpa Paramater

In [18]:
# Initialize Naive Bayes classifier
svm_tanpa_parameter = SVC(kernel='linear', random_state=42)

# Train and evaluate on Bag of Words (BoW) features
bow_accuracy, bow_f1,bow_report,bow_matrix = train_and_evaluate(svm_tanpa_parameter , count_train_features, y_train_encoded, count_test_features, y_test_encoded)

In [19]:
print("Bag of Words (BoW) Features:")
print("Accuracy: {:.2f}%".format(bow_accuracy))
print("F1 Score: {:.2f}%".format(bow_f1))
print("Classification Report:\n", bow_report)
print("Confusion Matrix:\n", bow_matrix)

Bag of Words (BoW) Features:
Accuracy: 90.22%
F1 Score: 90.19%
Classification Report:
               precision    recall  f1-score   support

     negatif       0.89      0.87      0.88       313
      netral       0.86      0.98      0.92       291
     positif       0.96      0.86      0.91       296

    accuracy                           0.90       900
   macro avg       0.91      0.90      0.90       900
weighted avg       0.91      0.90      0.90       900

Confusion Matrix:
 [[272  30  11]
 [  7 284   0]
 [ 25  15 256]]


#### Tuning

In [21]:
# # Define the parameter grid
# param_grid_svm = {'C': [0.1, 1, 10],
#                   'gamma': [0.1, 0.01, 0.001],
#                   'kernel': ['linear', 'rbf', 'sigmoid']}

# # Initialize GridSearchCV
# grid_search_svm = GridSearchCV(svm_tanpa_parameter, param_grid_svm, cv=5, scoring='accuracy')

# # Fit the grid search to the data
# grid_search_svm.fit(count_train_features, y_train_encoded)

# # Get the best parameters
# best_params_svm = grid_search_svm.best_params_
# print("Best Parameters:", best_params_svm)

# # Get the best model
# best_model_svm = grid_search_svm.best_estimator_
# print("Best Parameters:", best_params_svm)

Best Parameters: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
Best Parameters: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}


#### Pakai Parameter

In [26]:
# Initialize Naive Bayes classifier
svm_parameter = SVC(C=10, gamma=0.001, kernel='rbf')

# Train and evaluate on Bag of Words (BoW) features
bow_accuracy, bow_f1,bow_report,bow_matrix = train_and_evaluate(svm_parameter , count_train_features, y_train_encoded, count_test_features, y_test_encoded)


In [27]:
print("Bag of Words (BoW) Features:")
print("Accuracy: {:.2f}%".format(bow_accuracy))
print("F1 Score: {:.2f}%".format(bow_f1))
print("Classification Report:\n", bow_report)
print("Confusion Matrix:\n", bow_matrix)

Bag of Words (BoW) Features:
Accuracy: 90.44%
F1 Score: 90.40%
Classification Report:
               precision    recall  f1-score   support

     negatif       0.92      0.87      0.89       313
      netral       0.86      0.99      0.92       291
     positif       0.95      0.86      0.90       296

    accuracy                           0.90       900
   macro avg       0.91      0.91      0.90       900
weighted avg       0.91      0.90      0.90       900

Confusion Matrix:
 [[272  27  14]
 [  4 287   0]
 [ 21  20 255]]


### Model Logistic Regression

In [28]:
from sklearn.linear_model import LogisticRegression


#### Tanpa Paramater

In [29]:
# Initialize Naive Bayes classifier
lr_tanpa_parameter = LogisticRegression(random_state=42,max_iter=1000)

# Train and evaluate on Bag of Words (BoW) features
bow_accuracy, bow_f1,bow_report,bow_matrix = train_and_evaluate(lr_tanpa_parameter , count_train_features, y_train_encoded, count_test_features, y_test_encoded)


In [30]:
print("Bag of Words (BoW) Features:")
print("Accuracy: {:.2f}%".format(bow_accuracy))
print("F1 Score: {:.2f}%".format(bow_f1))
print("Classification Report:\n", bow_report)
print("Confusion Matrix:\n", bow_matrix)

Bag of Words (BoW) Features:
Accuracy: 92.22%
F1 Score: 92.20%
Classification Report:
               precision    recall  f1-score   support

     negatif       0.93      0.89      0.91       313
      netral       0.89      0.98      0.93       291
     positif       0.96      0.90      0.92       296

    accuracy                           0.92       900
   macro avg       0.92      0.92      0.92       900
weighted avg       0.92      0.92      0.92       900

Confusion Matrix:
 [[279  22  12]
 [  5 286   0]
 [ 16  15 265]]


#### Tuning

In [84]:
# # Define the parameter grid
# param_grid_lr = {'C': [0.001, 0.01, 0.1, 1, 10],
#                   'max_iter': [1000]
#                  }

# # Initialize GridSearchCV
# grid_search_lr = GridSearchCV(lr_tanpa_parameter, param_grid_lr, cv=5, scoring='accuracy')

# # Fit the grid search to the data
# grid_search_lr.fit(count_train_features, y_train_encoded)

# # Get the best parameters
# best_params_lr = grid_search_lr.best_params_
# print("Best Parameters:", best_params_lr)

# # Get the best model
# best_model_lr = grid_search_lr.best_estimator_
# print("Best Parameters:", best_params_lr)


#### Parameter

In [31]:
# Initialize Naive Bayes classifier
lr_parameter =  LogisticRegression(C=0.1,max_iter=1000)

## Train and evaluate on Bag of Words (BoW) features
bow_accuracy, bow_f1,bow_report,bow_matrix = train_and_evaluate(lr_parameter , count_train_features, y_train_encoded, count_test_features, y_test_encoded)


In [32]:
print("Bag of Words (BoW) Features:")
print("Accuracy: {:.2f}%".format(bow_accuracy))
print("F1 Score: {:.2f}%".format(bow_f1))
print("Classification Report:\n", bow_report)
print("Confusion Matrix:\n", bow_matrix)

Bag of Words (BoW) Features:
Accuracy: 92.44%
F1 Score: 92.43%
Classification Report:
               precision    recall  f1-score   support

     negatif       0.95      0.88      0.92       313
      netral       0.87      0.99      0.93       291
     positif       0.96      0.91      0.93       296

    accuracy                           0.92       900
   macro avg       0.93      0.93      0.92       900
weighted avg       0.93      0.92      0.92       900

Confusion Matrix:
 [[275  27  11]
 [  2 289   0]
 [ 11  17 268]]


### Model Random Forest

In [34]:
from sklearn.ensemble import RandomForestClassifier

#### Tanpa Paramater

In [35]:
# Initialize Naive Bayes classifier
rfc_tanpa_parameter = RandomForestClassifier(random_state=42)

# Train and evaluate on Bag of Words (BoW) features
bow_accuracy, bow_f1,bow_report,bow_matrix = train_and_evaluate(rfc_tanpa_parameter, count_train_features, y_train_encoded, count_test_features, y_test_encoded)


In [36]:
print("Bag of Words (BoW) Features:")
print("Accuracy: {:.2f}%".format(bow_accuracy))
print("F1 Score: {:.2f}%".format(bow_f1))
print("Classification Report:\n", bow_report)
print("Confusion Matrix:\n", bow_matrix)

Bag of Words (BoW) Features:
Accuracy: 90.11%
F1 Score: 89.99%
Classification Report:
               precision    recall  f1-score   support

     negatif       0.95      0.81      0.87       313
      netral       0.86      0.99      0.92       291
     positif       0.90      0.92      0.91       296

    accuracy                           0.90       900
   macro avg       0.90      0.90      0.90       900
weighted avg       0.91      0.90      0.90       900

Confusion Matrix:
 [[252  30  31]
 [  3 288   0]
 [  9  16 271]]


### Tuning

In [92]:
# # Define the parameter grid
# param_grid_rfc = {'n_estimators': [50, 100, 200],
#                   'max_depth': [None, 10, 20, 30],
#                   'min_samples_split': [2, 5, 10]}

# # Initialize GridSearchCV
# grid_search_rfc = GridSearchCV(rfc_tanpa_parameter, param_grid_rfc, cv=5, scoring='accuracy')

# # Fit the grid search to the data
# grid_search_rfc.fit(count_train_features, y_train_encoded)

# # Get the best parameters
# best_params_rfc = grid_search_rfc.best_params_
# print("Best Parameters:", best_params_rfc)

# # Get the best model
# best_model_rfc = grid_search_rfc.best_estimator_
# print("Best Parameters:", best_model_rfc)

#### Paramater

In [39]:
# Initialize Naive Bayes classifier
rfc_parameter = RandomForestClassifier(min_samples_split=2,n_estimators=100)

# Train and evaluate on Bag of Words (BoW) features
bow_accuracy, bow_f1,bow_report,bow_matrix = train_and_evaluate(rfc_tanpa_parameter, count_train_features, y_train_encoded, count_test_features, y_test_encoded)


In [40]:
print("Bag of Words (BoW) Features:")
print("Accuracy: {:.2f}%".format(bow_accuracy))
print("F1 Score: {:.2f}%".format(bow_f1))
print("Classification Report:\n", bow_report)
print("Confusion Matrix:\n", bow_matrix)

Bag of Words (BoW) Features:
Accuracy: 90.11%
F1 Score: 89.99%
Classification Report:
               precision    recall  f1-score   support

     negatif       0.95      0.81      0.87       313
      netral       0.86      0.99      0.92       291
     positif       0.90      0.92      0.91       296

    accuracy                           0.90       900
   macro avg       0.90      0.90      0.90       900
weighted avg       0.91      0.90      0.90       900

Confusion Matrix:
 [[252  30  31]
 [  3 288   0]
 [  9  16 271]]


### Model Ensemble Learning

In [96]:
# Mengambil model terbaik dari hasil tuning
best_model_naive_bayes = model_naiye_parameter
best_model_svm = svm_parameter
best_model_random_forest = rfc_parameter
best_model_logistic_regression = lr_parameter

In [97]:
from sklearn.ensemble import VotingClassifier

#### Model Gabungin 4 Model

In [98]:
# # Membuat model ensemble
# ensemble_model_4 = VotingClassifier(estimators=[
#     ('naive_bayes', best_model_naive_bayes),
#     ('svm', best_model_svm),
#     ('random_forest', best_model_random_forest),
#     ('logistic_regression', best_model_logistic_regression)
# ], voting='hard')

# # Train and evaluate on Bag of Words (BoW) features
# bow_accuracy, bow_f1,bow_report,bow_matrix = train_and_evaluate(ensemble_model_4, count_train_features, y_train_encoded, count_test_features, y_test_encoded)

In [99]:
# print("Bag of Words (BoW) Features:")
# print("Accuracy: {:.2f}%".format(bow_accuracy))
# print("F1 Score: {:.2f}%".format(bow_f1))
# print("Classification Report:\n", bow_report)
# print("Confusion Matrix:\n", bow_matrix)

Bag of Words (BoW) Features:
Accuracy: 94.32%
F1 Score: 94.31%
Classification Report:
               precision    recall  f1-score   support

     negatif       0.95      0.91      0.93       272
      netral       0.91      0.99      0.95       280
     positif       0.97      0.93      0.95       276

    accuracy                           0.94       828
   macro avg       0.94      0.94      0.94       828
weighted avg       0.94      0.94      0.94       828

Confusion Matrix:
 [[247  19   6]
 [  3 276   1]
 [ 11   7 258]]


#### Model 3 dengan Akurasi Terbaik

#### Model 2 Dengan Nilai Akurasi Terbaik 

In [107]:
import joblib
# # Dictionary to store model and vectorizer
# model_data = {
#     'model': ensemble_model_4,  # your trained Logistic Regression model
#     'vectorizer': count_vectorizer,
#     'label_encoder': label_encoder
# }

# # Save the model data to a file
# joblib.dump(model_data, 'gabungan_4model.pkl')

In [108]:
# # Later, to load the model data from the file
# loaded_model_data = joblib.load('gabungan_4model.pkl')

# # Extract model and vectorizer from loaded data
# loaded_model = loaded_model_data['model']
# loaded_vectorizer = loaded_model_data['vectorizer']
# new_data = ["bogor keluarga fw 22 perempuan dibunuh pacarnya ra alias alung 20 bogor pelaku dihukum berat ayah korban iwan irawan terima alung dihukum 15 tahun penjara sesuai pasal 338 kuhp pembunuhan iwan berharap alung dihukum seberat beratnya penjara seumur hidup baca wanita dibunuh pacarnya bogor mulutnya dibekap hidungnya digigit kepolisian hukum seberat beratnya seadil adilnya nyawa anak keberatan 15 tahun harapannya seumur hidup iwan ditemui mapolresta bogor kota selasa 5 12 2023 iwan menyangka kekasih anaknya pelaku pembunuh darah dagingnya hubungan asmara terjalin tahun setahun kenal dianggap anak main rumah iwan timbal baliknya bohong anak meninggal jatuh motor dibunuh sambung baca bekap pacar tewas bogor pelaku niat membunuhsejak peristiwa tragis iwan keluarga tersangka iktikad pemakaman anaknya keluarga tersangka keluarga tersangka iktikad baiknya ga komunikasinya iwan nyawa aja pertemuan pemakaman sambung", 
#             "calon presiden bacapres ganjar pranowo mengawali aktivitasnya berlari seputaran kota bandung jawa barat pria berambut putih menyusuri gang gang pemukiman warga"]

# # Extract model, vectorizer, and label encoder from loaded data
# loaded_model = loaded_model_data['model']
# loaded_vectorizer = loaded_model_data['vectorizer']
# loaded_label_encoder = loaded_model_data['label_encoder']

# # Vectorize the new data using the loaded CountVectorizer
# new_data_features = loaded_vectorizer.transform(new_data)

# # Predict sentiment using the loaded model
# predictions = loaded_model.predict(new_data_features)

# # Decode numerical labels back to text labels using the loaded LabelEncoder
# predicted_sentiments = loaded_label_encoder.inverse_transform(predictions)

# # Print the predicted sentiments
# for text, sentiment in zip(new_data, predicted_sentiments):
#     print(f"Text: '{text}' -> Predicted Sentiment: '{sentiment}'")

Text: 'bogor keluarga fw 22 perempuan dibunuh pacarnya ra alias alung 20 bogor pelaku dihukum berat ayah korban iwan irawan terima alung dihukum 15 tahun penjara sesuai pasal 338 kuhp pembunuhan iwan berharap alung dihukum seberat beratnya penjara seumur hidup baca wanita dibunuh pacarnya bogor mulutnya dibekap hidungnya digigit kepolisian hukum seberat beratnya seadil adilnya nyawa anak keberatan 15 tahun harapannya seumur hidup iwan ditemui mapolresta bogor kota selasa 5 12 2023 iwan menyangka kekasih anaknya pelaku pembunuh darah dagingnya hubungan asmara terjalin tahun setahun kenal dianggap anak main rumah iwan timbal baliknya bohong anak meninggal jatuh motor dibunuh sambung baca bekap pacar tewas bogor pelaku niat membunuhsejak peristiwa tragis iwan keluarga tersangka iktikad pemakaman anaknya keluarga tersangka keluarga tersangka iktikad baiknya ga komunikasinya iwan nyawa aja pertemuan pemakaman sambung' -> Predicted Sentiment: 'negatif'
Text: 'calon presiden bacapres ganjar p